<a href="https://colab.research.google.com/github/atulghunavat/atul_INFO5731_Spring2020/blob/master/Assignment%202/INFO5731_Assignment_Two_Atul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [2019 Dell labtop](https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/dp/B07N49F51N/ref=sr_1_11?crid=1IJ7UWF2F4GHH&keywords=dell%2Bxps%2B15&qid=1580173569&sprefix=dell%2Caps%2C181&sr=8-11&th=1) on amazon.

(2) Collect the top 100 User Reviews of the film [Joker](https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv) from IMDB.

(3) Collect the abstracts of the top 100 research papers by using the query [natural language processing](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(4) Collect the top 100 tweets by using hashtag ["#wuhancoronovirus"](https://twitter.com/hashtag/wuhancoronovirus) from Twitter. 


In [0]:
!pip install Scrapy
!pip3 install AllenNLP

In [0]:
!scrapy runspider /content/amazon_review.py -o amazonreviews.csv

In [0]:
# Below code is saved in file amazon_review.py which is used in the above terminal statement to generate csv file
# Code for question 1
'''
import scrapy
from scrapy.http import Request

# Creating a new class to implement Spide
class AmazonReviewsSpider(scrapy.Spider):
    
    custom_settings = {
      'CONCURRENT_REQUESTS':1,
      'DOWNLOAD_DELAY': 5
    }
    # Spider name
    name = 'amazon_reviews'
     
    # Domain names to scrape
    allowed_domains = ["amazon.com"]
     
    # Base URL for the MacBook air reviews
    myBaseUrl = "https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/product-reviews/B07N49F51N/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber="
    start_urls= []
    
    # Creating list of urls to be scraped by appending page number a the end of base url
    for i in range(1,9):
        start_urls.append(myBaseUrl+str(i))

    def start_requests(self):
        for url in self.start_urls:
            print(url)
            yield Request(url=url, callback=self.parse_data, dont_filter=True)

    def parse_data(self, response):
      data = response.css('#cm_cr-review_list')
      names = data.css('.a-profile-name')
      stars = data.css('.review-rating')
      reviewTitles = data.css('.review-title')
      reviews = data.css('.review-text')
      count = 0

      # Combining the results
      for user in names:
          yield {'Name': ''.join(user.xpath('.//text()').extract()),
                 'Rating': ''.join(stars[count].xpath('.//text()').extract()),
                 'Titles': ''.join(reviewTitles[count].xpath('.//text()').extract()),
                 'Reviews': ''.join(reviews[count].xpath(".//text()").extract())
                 }
          count = count + 1
'''

# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [127]:
# Code for question 2

import pandas as pd
import nltk
import string
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re

review_df = pd.read_csv('/content/amazonreviews.csv')
review_df['Cleaned'] = review_df['Reviews']
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
regex = re.compile('[^a-zA-Z\s]')

#This method removes numbers, punctuation and special characters.
def remove_noise_special_numbers(text):
    return re.sub(regex, '', text)

#This menthod removes stop words
def remove_stopwords(text):
  words = " ".join([word for word in text.split() if word not in stopwords.words('english')])
  return words

#This method performs stemming
def perform_stemming(text):
  stem = " ".join([stemmer.stem(i) for i in text.split()])
  return stem

#This method performs lemmitization
def perform_lemmatization(text):
  lem = " ".join([lemmatizer.lemmatize(i) for i in text.split()])
  return lem


review_df['Cleaned'] = review_df['Cleaned'].apply(lambda x: remove_noise_special_numbers(x))
review_df['Cleaned'] = review_df['Cleaned'].apply(lambda x: x.lower())
review_df['Cleaned'] = review_df['Cleaned'].apply(lambda x: remove_stopwords(x))
review_df['Cleaned'] = review_df['Cleaned'].apply(lambda x: perform_stemming(x))
review_df['Cleaned'] = review_df['Cleaned'].apply(lambda x: perform_lemmatization(x))

review_df.to_csv('/content/amazonreviews.csv')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [140]:
# Question 3.1 - Parts of Speech (POS) Tagging

import spacy
import nltk
import en_core_web_sm
from spacy import displacy
from collections import Counter
nltk.download('averaged_perceptron_tagger')

review_df = pd.read_csv('/content/amazonreviews.csv')

all_sentence = ""
for row in review_df.itertuples():
  sentence = row.Cleaned + " "
  all_sentence = all_sentence + sentence

nltk_pos_tagged = nltk.pos_tag(all_sentence.split())
pd.DataFrame(nltk_pos_tagged, columns=['Word', 'POS tag'])
all_text_counts = Counter(tag[0] for word,tag in nltk_pos_tagged)
print ('Total number of Nouns in the text:' + str(all_text_counts['N']))
print ('Total number of Verbs in the text:' + str(all_text_counts['V']))
print ('Total number of Adjectives in the text:' + str(all_text_counts['J']))
print ('Total number of Adverbs in the text:' + str(all_text_counts['R']))
print ('POS Tagging for all the words in the text:')
for x in nltk_pos_tagged:
  print(x)



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Total number of Nouns in the text:773
Total number of Verbs in the text:196
Total number of Adjectives in the text:393
Total number of Adverbs in the text:110
POS Tagging for all the words in the text:
('bought', 'VBN')
('comput', 'NN')
('look', 'NN')
('comput', 'VBD')
('wife', 'NN')
('ssd', 'NN')
('least', 'JJS')
('gb', 'JJ')
('ram', 'NN')
('least', 'JJS')
('processor', 'NN')
('also', 'RB')
('wife', 'NN')
('feel', 'VB')
('comfort', 'NN')
('work', 'NN')
('windowsthi', 'IN')
('one', 'CD')
('good', 'JJ')
('balanc', 'NN')
('price', 'NN')
('spec', 'NNS')
('look', 'VBP')
('lot', 'NN')
('like', 'IN')
('macbook', 'NN')
('pro', 'FW')
('feel', 'FW')
('littl', 'FW')
('fragil', 'NN')
('mac', 'NN')
('also', 'RB')
('like', 'IN')
('one', 'CD')
('full', 'JJ')
('keyboard', 'NN')
('includ', 'NN')
('number', 'NN')
('ke

In [0]:
from allennlp.predictors.predictor import Predictor
entity_predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/fine-grained-ner-model-elmo-2018.12.21.tar.gz")
constituency_predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/elmo-constituency-parser-2018.03.14.tar.gz")
dependency_predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/biaffine-dependency-parser-ptb-2018.08.23.tar.gz")


In [130]:
# Question 3.2 Dependency Parsing
# Using AllenNLP

from nltk.tree import *
for row in review_df.itertuples():
  sent1 = row.Cleaned
  dependency_results = dependency_predictor.predict(sentence=sent1)
  root = (dependency_results['hierplane_tree']['root'])
  print(root)


{'word': 'bought', 'nodeType': 'root', 'attributes': ['VBD'], 'link': 'root', 'spans': [{'start': 0, 'end': 7}], 'children': [{'word': 'look', 'nodeType': 'iobj', 'attributes': ['NN'], 'link': 'iobj', 'spans': [{'start': 14, 'end': 19}]}, {'word': 'ssd', 'nodeType': 'dobj', 'attributes': ['NN'], 'link': 'dobj', 'spans': [{'start': 31, 'end': 35}], 'children': [{'word': 'comput', 'nodeType': 'nn', 'attributes': ['NN'], 'link': 'nn', 'spans': [{'start': 7, 'end': 14}]}, {'word': 'comput', 'nodeType': 'nn', 'attributes': ['NN'], 'link': 'nn', 'spans': [{'start': 19, 'end': 26}]}, {'word': 'wife', 'nodeType': 'nn', 'attributes': ['NN'], 'link': 'nn', 'spans': [{'start': 26, 'end': 31}]}, {'word': 'least', 'nodeType': 'amod', 'attributes': ['JJS'], 'link': 'amod', 'spans': [{'start': 35, 'end': 41}]}, {'word': 'gb', 'nodeType': 'nn', 'attributes': ['NNS'], 'link': 'nn', 'spans': [{'start': 41, 'end': 44}]}, {'word': 'ram', 'nodeType': 'dep', 'attributes': ['NN'], 'link': 'dep', 'spans': [{'

In [145]:
# Question 3.2 Constituency Parsing

from nltk.tree import *
for row in review_df.itertuples():
  sent1 = row.Cleaned  
  constituency_results = constituency_predictor.predict_json({"sentence":sent1})
  tree = Tree.fromstring(constituency_results['trees'])
  tree.pretty_print()


                                                                                                                                                                                                     SINV                                                                                                                                                                                                                               
   ___________________________________________________________________________________________________________________________________________________________________________________________________|______________________________________________________________________________________________________________________________________________________________________________________________________________________________    
  |                                                                                          NP                                                      

In [0]:
# Question 3.3 Named Entity Recognation
# Using AllenNLP. No output or entity recognition was not successful.

def process_allen_results(allen_results):
  ents = set()
  for word, tag in zip(allen_results["words"], allen_results["tags"]):
    if tag != "O":
      ent_position, ent_type = tag.split("-")
      if ent_position == "U":
        ents.add((word,ent_type))
      else:
        if ent_position == "B":
          w = word
        elif ent_position == "I":
          w += " " + word
        elif ent_position == "L":
          w += " " + word
          ents.add((w,ent_type))
  return ents

for row in review_df.itertuples():
  ner_results = entity_predictor.predict(sentence=row.Cleaned)
  result = process_allen_results(ner_results)
  if(len(result) > 0):
      print(result)
   

In [161]:
# Since AllenNLP did not give any results I used Spacy for NER and the count is also provided.

nlp = en_core_web_sm.load()
doc = nlp(all_sentence)

all_count = Counter(ent.label_ for ent in doc.ents)
name_count = all_count['PERSON']
date_count = all_count['DATE']
org_count = all_count['ORG']
loc_count = all_count['LOC']
prod_count = all_count['PRODUCT']

print('Number of date entity:' + str(date_count))
print('Number of person name entity:' + str(name_count))
print('Number of organizations entity:' + str(org_count))
print('Number of locations entity:' + str(loc_count))
print('Number of product name entity:' + str(prod_count))

for ent in doc.ents:
  if(ent.label_ == 'DATE' or ent.label_ == 'ORG' or ent.label_ == 'PRODUCT' or ent.label_ == 'LOC' or ent.label_ == 'PERSON'):
    print(ent.text, ent.label_)


Number of date entity:6
Number of person name entity:0
Number of organizations entity:0
Number of locations entity:0
Number of product name entity:0
one month DATE
month DATE
far week DATE
one month DATE
birthday week DATE
two day DATE


**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

Review: "Excellent PC. Works perfectly, is VERY fast. It was worth the money."

Cleaned Review: "excel pc work perfectli fast worth money"

**Dependency Tree:**

{'word': 'excel', 'nodeType': 'root', 'attributes': ['VB'], 'link': 'root', 'spans': [{'start': 0, 'end': 6}], 'children': [{'word': 'perfectli', 'nodeType': 'dobj', 'attributes': ['NN'], 'link': 'dobj', 'spans': [{'start': 14, 'end': 24}], 'children': [{'word': 'pc', 'nodeType': 'amod', 'attributes': ['JJ'], 'link': 'amod', 'spans': [{'start': 6, 'end': 9}]}, {'word': 'work', 'nodeType': 'nn', 'attributes': ['NN'], 'link': 'nn', 'spans': [{'start': 9, 'end': 14}]}, {'word': 'worth', 'nodeType': 'amod', 'attributes': ['JJ'], 'link': 'amod', 'spans': [{'start': 29, 'end': 35}], 'children': [{'word': 'money', 'nodeType': 'dep', 'attributes': ['NN'], 'link': 'dep', 'spans': [{'start': 35, 'end': 41}]}]}]}, {'word': 'fast', 'nodeType': 'advmod', 'attributes': ['RB'], 'link': 'advmod', 'spans': [{'start': 24, 'end': 29}]}]}

Explanation: 

In the above dependency parsing,
the word "excel" is the centre of the sentence and is represented as the root. This root is linked to two nodes "perfectli" and "fast" by dobj (direct object) and advmod (adverbial modifier) respectively. "perfectli" is linked to three nodes "pc", "work" and "worth" by amod(adjectival modifier), nn(nominal) and amod respectively. "money" is the leaf node connected to "worth" by dep(indeterminate). In this tree work is considered as a subject of importance and words like pc, worth provide modification to the meaning of the noun.

In [162]:
# Constituency tree explanation
from nltk.tree import *
constituency_results = constituency_predictor.predict_json({"sentence":"excel pc work perfectli fast worth money"})
tree = Tree.fromstring(constituency_results['trees'])
tree.pretty_print()

                        S                     
                        |                      
                        VP                    
   _____________________|__________________    
  |         NP          |      |     |     |  
  |     ____|___        |      |     |     |   
  |   ADJP      |       NP     |     |     NP 
  |    |        |       |      |     |     |   
  VB   JJ       NN      NN     RB    JJ    NN 
  |    |        |       |      |     |     |   
excel  pc      work perfectli fast worth money



**Constituency Parsing:**

This parsing extract the syntantic structure of a sentence. Each sentence can be parsed into a noun and a verb phrase. In the above sentence Verb Phrase perfectly is labeled as a NN. Also "money" and "work" is identified as subjects of the sentence. "excel" is identified as a verb phrase.